### Install requirements

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt -U

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Get HTML content

In [2]:
import requests

In [3]:
response = requests.get('https://www.premierinjuries.com/injury-table.php')

In [4]:
response.status_code

200

### Start parsing

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content)

In [47]:
from dataclasses import dataclass

@dataclass
class Player:
    name: str
    reason: str
    
@dataclass
class Team:
    name: str
    inj_count: int
    team_id: str
    players: list[Player]

In [65]:
results = []

table = soup.find('table', class_='injury-table')
teams = table.find_all('tr', class_='heading')

for team in teams:
    
    team_id = team['data-team-id']
    
    title_team = team.find('div', class_='injury-team').text
    
    count_players = int(team.find('span', class_='injury-count2-num').text)
    
    results.append(Team(title_team, count_players, team_id, players=[]))

for team in results:
    
    players = table.find_all('tr', f'player-row team_{team.team_id}')
    
    for player in players:
        player_name = player.find('div', class_='mob-title', text='Player').parent.text[6:] 
        reason = player.find('div', class_='mob-title', text='Reason').parent.text[6:]
        
        team.players.append(Player(player_name, reason))
        
for team in results:
    print(f'{team.name} - {team.inj_count}')  
    for player in team.players:
        print(f'{player.name} - {player.reason}')
        
    print('--------------------')

AFC Bournemouth - 8
David Brooks - Thigh Injury
Marcus Tavernier - Thigh Injury
Dominic Solanke - Knee Injury
Lewis Cook - Knee Injury
Lloyd Kelly - Other
Jack Stephens - Other
Ryan Fredericks  - Other
Junior Stanislas - Other
--------------------
Arsenal - 4
Gabriel Fernando de Jesus - Knee Injury
Reiss Nelson - Thigh Injury
Emile Smith Rowe - Thigh Injury
Mohamed Elneny - Knee Injury
--------------------
Aston Villa - 3
Diego Carlos Santos Silva - Calf/Shin/Heel Injury
Bertrand Isidore Traore - Knee Injury
Jed Steer  - Calf/Shin/Heel Injury
--------------------
Brentford - 3
Pontus Jansson - Thigh Injury
Shandon Baptiste - Groin/Hip/Pelvic Injury
Frank Onyeka - Thigh Injury
--------------------
Brighton & Hove Albion - 6
Jakub Moder - Knee Injury
Levi Colwill - Other
Alexis Mac Allister - Suspended
Adam Lallana  - Other
Evan Ferguson - Knee Injury
Enock Mwepu - Chest/Abdominal Injury
--------------------
Chelsea - 8
Armando Broja - Knee Injury
Christian Pulisic - Knee Injury
N'Golo K